In [ ]:
reset -sf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -r /content/Math_solving_using_LLM
!git clone https://github.com/dunp7/Math_solving_using_LLM.git

Cloning into 'Math_solving_using_LLM'...
remote: Enumerating objects: 345, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 345 (delta 84), reused 81 (delta 38), pack-reused 216 (from 1)
Receiving objects: 100% (345/345), 982.29 KiB | 9.54 MiB/s, done.
Resolving deltas: 100% (237/237), done.


In [ ]:
!pip install huggingface_hub
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install -r /content/Math_solving_using_LLM/requirements.txt

  Using cached torchmetrics-1.7.1-py3-none-any.whl.metadata (21 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached torchmetrics-1.7.1-py3-none-any.whl (961 kB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.3.0.75
    Uninstalling nvidia-cudnn-cu12-9.3.0.75:
      Successfully uninstalled nvidia-cudnn-cu12-9.3.0.75
  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83


In [ ]:
import random
import torch
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_from_disk
from huggingface_hub import login
import sys
import os
sys.path.append("/content/Math_solving_using_LLM")
sys.path.append("/content/Math_solving_using_LLM/my_utils")


from my_utils.uncertainty_measures.p_true import generate_p_true
from my_utils.uncertainty_measures.semantic_entropy import is_entailment_transformer,generate_SE
from my_utils.uncertainty_measures.PE import generate_PE

token_comp = "your_hugging_face_token"
login(token= token_comp)


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
seed = 42
random.seed(seed)

cpu


# Load Data

In [ ]:
# LLM - Falcon_instruct(7B), Mistral(7B, Instruct)

### Generate answers LLM
models = {
    "gemma2_9b": "google/gemma-2-9b-it",
    "gemma2_2b": "google/gemma-2-2b-it",
    "llama2_7b": "meta-llama/Llama-2-7b-chat-hf",
    "mistral7b_instruct": "mistralai/Mistral-7B-Instruct-v0.2",
}

# Prompts
prompts = {
    "default": None,
    "zero_shot_CoT": "Let’s think step by step.",
    "ite_ape": "We’ll approach this methodically, proceeding one step at a time.",
    # "apo": "Carefully analyze the details and perform precise arithmetic operations step by step, ensuring to apply the correct mathematical principles for accurate calculations.",
    # "pe2": "Carefully calculate step by step, considering all details, including unique counts and overlaps. Accurately apply arithmetic to find the numerical answer, ensuring logical operations are correctly followed."
}


In [ ]:
import os

base_path = "/content/drive/MyDrive/Math Solving Problem/Result/measures"
# base_path = "/content/drive/MyDrive/hallucination_data"

def take_data(base_path, models, prompts ,dataset_name):

  # Create variables
  data = {model: {} for model in models.keys()}


  # Process each model
  for model, _ in models.items():
      print(f"Processing model: {model}")

      data[model] = {prompt: [] for prompt in prompts.keys()}

      # Process each prompt
      for prompt in prompts.keys():
          # Retrieve data
          data_path = os.path.join(base_path, model, prompt, dataset_name)
          # print(data_path)
          if os.path.exists(data_path):
              result = load_from_disk(data_path)
          else:
              print(f" No data found at {data_path}.")

          # Check if data is valid
          if data is not None:
              data[model][prompt].append(result)
          else:
              continue
  return data

In [ ]:
# se_result = take_data(base_path+"SE", models ,prompts, 'svamp')
pe_result = take_data(base_path+"/PE", models ,prompts, 'svamp')
p_true_result = take_data(base_path+"/p_true", models ,prompts, 'svamp')
se_result = take_data(base_path+"/SE", models ,prompts, 'svamp')


Processing model: gemma2_9b
Processing model: gemma2_2b
Processing model: llama2_7b
Processing model: mistral7b_instruct
Processing model: gemma2_9b
Processing model: gemma2_2b
Processing model: llama2_7b
Processing model: mistral7b_instruct
Processing model: gemma2_9b
Processing model: gemma2_2b
Processing model: llama2_7b
Processing model: mistral7b_instruct


# Evaluate Metrics

In [ ]:
import pandas as pd
def cal_metric(data, metric_cal_func, name, label_type):
    """
    Calculate metrics for each dataset and prompt and store the results in a DataFrame.

    :param data: Dictionary containing model, prompt, and dataset information.
    :param metric_cal_func: Function to calculate metrics (e.g., AUROC).
    :return: DataFrame containing the results.
    """
    results = []

    for model, prompts_data in data.items():
        for prompt, dataset in prompts_data.items():
            if dataset:
                auroc_value = metric_cal_func([dataset[0]], name, "/content/results/", label_type)
                results.append({
                    "Model": model,
                    "Prompt": prompt,
                    "label": label_type,
                    "Metric": name,
                    "Value": auroc_value[0]

                })

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
from my_utils.metrics import calculate_auroc, calculate_aurac

In [ ]:
pe1 = cal_metric(pe_result, calculate_auroc, 'PE', "SQuAD")
pe2 = cal_metric(pe_result, calculate_auroc, 'PE', "LLM")
lnpe1 = cal_metric(pe_result, calculate_auroc, 'Ln-PE', "SQuAD")
lnpe2 = cal_metric(pe_result, calculate_auroc, 'Ln-PE', "LLM")
ptrue1 = cal_metric(p_true_result, calculate_auroc, 'p_true', "SQuAD")
ptrue2 = cal_metric(p_true_result, calculate_auroc, 'p_true', "LLM")
se1 = cal_metric(se_result, calculate_auroc, 'SE', "SQuAD")
se2 = cal_metric(se_result, calculate_auroc, 'SE', "LLM")

svamp-PE                   SQuAD      dataset:   0.4590
svamp-PE                   SQuAD      dataset:   0.5855
svamp-PE                   SQuAD      dataset:   0.6122
svamp-PE                   SQuAD      dataset:   0.4878
svamp-PE                   SQuAD      dataset:   0.4978
svamp-PE                   SQuAD      dataset:   0.4680
svamp-PE                   SQuAD      dataset:   0.3782
svamp-PE                   SQuAD      dataset:   0.4633
svamp-PE                   SQuAD      dataset:   0.5098
svamp-PE                   SQuAD      dataset:   0.3539
svamp-PE                   SQuAD      dataset:   0.4529
svamp-PE                   SQuAD      dataset:   0.4803
svamp-PE                   LLM        dataset:   0.5178
svamp-PE                   LLM        dataset:   0.1856
svamp-PE                   LLM        dataset:   0.5311
svamp-PE                   LLM        dataset:   0.5195
svamp-PE                   LLM        dataset:   0.4402
svamp-PE                   LLM        dataset:  

In [ ]:
merged_results = pd.concat([pe1, pe2, lnpe1, lnpe2, ptrue1, ptrue2,se1, se2], ignore_index=True)
merged_results

,Model,Prompt,label,Metric,Value
0,gemma2_9b,default,SQuAD,PE,0.459048
1,gemma2_9b,zero_shot_CoT,SQuAD,PE,0.585475
2,gemma2_9b,ite_ape,SQuAD,PE,0.612184
3,gemma2_2b,default,SQuAD,PE,0.487756
4,gemma2_2b,zero_shot_CoT,SQuAD,PE,0.497778
...,...,...,...,...,...
91,llama2_7b,zero_shot_CoT,LLM,SE,0.560000
92,llama2_7b,ite_ape,LLM,SE,0.464286
93,mistral7b_instruct,default,LLM,SE,0.656357
94,mistral7b_instruct,zero_shot_CoT,LLM,SE,0.709474


In [ ]:
output_file = "AUROC.xlsx"
merged_results.to_excel(output_file, index=False)

In [ ]:
pe1 = cal_metric(pe_result, calculate_aurac, 'PE', "SQuAD")
pe2 = cal_metric(pe_result, calculate_aurac, 'PE', "LLM")
lnpe1 = cal_metric(pe_result, calculate_aurac, 'Ln-PE', "SQuAD")
lnpe2 = cal_metric(pe_result, calculate_aurac, 'Ln-PE', "LLM")
ptrue1 = cal_metric(p_true_result, calculate_aurac, 'p_true', "SQuAD")
ptrue2 = cal_metric(p_true_result, calculate_aurac, 'p_true', "LLM")
se1 = cal_metric(se_result, calculate_aurac, 'SE', "SQuAD")
se2 = cal_metric(se_result, calculate_aurac, 'SE', "LLM")


svamp-PE                   dataset:   0.6859
svamp-PE                   dataset:   0.4522
svamp-PE                   dataset:   0.5024
svamp-PE                   dataset:   0.4930
svamp-PE                   dataset:   0.5120
svamp-PE                   dataset:   0.4882
svamp-PE                   dataset:   0.4106
svamp-PE                   dataset:   0.3625
svamp-PE                   dataset:   0.3324
svamp-PE                   dataset:   0.5928
svamp-PE                   dataset:   0.5766
svamp-PE                   dataset:   0.5207
svamp-PE                   dataset:   0.0778
svamp-PE                   dataset:   0.0546
svamp-PE                   dataset:   0.0819
svamp-PE                   dataset:   0.0473
svamp-PE                   dataset:   0.0755
svamp-PE                   dataset:   0.0517
svamp-PE                   dataset:   0.0375
svamp-PE                   dataset:   0.0389
svamp-PE                   dataset:   0.0064
svamp-PE                   dataset:   0.0101
svamp-PE  

In [ ]:
merged_results = pd.concat([pe1, pe2, lnpe1, lnpe2, ptrue1, ptrue2,se1, se2], ignore_index=True)
merged_results

,Model,Prompt,label,Metric,Value
0,gemma2_9b,default,SQuAD,PE,0.685921
1,gemma2_9b,zero_shot_CoT,SQuAD,PE,0.452244
2,gemma2_9b,ite_ape,SQuAD,PE,0.502444
3,gemma2_2b,default,SQuAD,PE,0.492987
4,gemma2_2b,zero_shot_CoT,SQuAD,PE,0.511953
...,...,...,...,...,...
91,llama2_7b,zero_shot_CoT,LLM,SE,0.043290
92,llama2_7b,ite_ape,LLM,SE,0.027192
93,mistral7b_instruct,default,LLM,SE,0.017069
94,mistral7b_instruct,zero_shot_CoT,LLM,SE,0.032861


In [ ]:
output_file = "AURAC.xlsx"
merged_results.to_excel(output_file, index=False)

# Analyze Accuracy Approaches

In [ ]:
data = take_data("/content/drive/MyDrive/Math Solving Problem/Result/labels", models ,prompts, 'svamp')


Processing model: gemma2_9b
Processing model: gemma2_2b
Processing model: llama2_7b
Processing model: mistral7b_instruct


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(models['mistral7b_instruct'])
test = data['mistral7b_instruct']['default'][1]['generated_answer_acc'][0]['sequences'][0]
tokenizer.decode(test)

IndexError: list index out of range